# Gesture Recognition

In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.


In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.


In [2]:
np.random.seed(2612)
import random as rn

rn.seed(2612)
from keras import backend as K
import tensorflow as tf

tf.random.set_seed(2612)

2024-03-02 08:16:22.441238: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 08:16:22.441273: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 08:16:22.442470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-02 08:16:22.448688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-02 08:16:25.072188: W tensorflow/compiler/tf2

In [3]:
tf.config.list_physical_devices("GPU")

2024-03-02 08:16:27.843257: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-02 08:16:27.924372: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-02 08:16:27.925065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.


In [4]:
train_doc = np.random.permutation(
    open(
        "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/train.csv"
    ).readlines()
)
val_doc = np.random.permutation(
    open(
        "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/val.csv"
    ).readlines()
)
batch_size = 45  # experiment with the batch size

## Generator

This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.


In [5]:
def generator(source_path, folder_list, batch_size, y, z):
    print("Source path = ", source_path, "; batch size =", batch_size)
    img_idx = [
        6,
        8,
        10,
        12,
        14,
        16,
        18,
        20,
        22,
        24,
    ]  # create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size  # calculate the number of batches
        for batch in range(num_batches):  # we iterate over the number of batches
            batch_data = np.zeros(
                (batch_size, len(img_idx), y, z, 3)
            )  # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros(
                (batch_size, 5)
            )  # batch_labels is the one hot representation of the output
            for folder in range(batch_size):  # iterate over the batch_size
                imgs = os.listdir(
                    source_path + "/" + t[folder + (batch * batch_size)].split(";")[0]
                )  # read all the images in the folder
                for idx, item in enumerate(
                    img_idx
                ):  #  Iterate iver the frames/images of a folder to read them in
                    image = imread(
                        source_path
                        + "/"
                        + t[folder + (batch * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    ).astype(np.float32)

                    # crop the images and resize them. Note that the images are of 2 different shape
                    # and the conv3D will throw error if the inputs in a batch have different shapes
                    resized_image = resize(image, (y, z))
                    normalized_image = resized_image / 255.0

                    batch_data[folder, idx, :, :, 0] = normalized_image[:, :, 0]
                    batch_data[folder, idx, :, :, 1] = normalized_image[:, :, 1]
                    batch_data[folder, idx, :, :, 2] = normalized_image[:, :, 2]

                batch_labels[
                    folder, int(t[folder + (batch * batch_size)].strip().split(";")[2])
                ] = 1
            yield batch_data, batch_labels  # you yield the batch_data and the batch_labels, remember what does yield do

        # write the code for the remaining data points which are left after full batches
        if len(folder_list) % batch_size != 0:
            remaining_batch_size = len(folder_list) % batch_size
            batch_data = np.zeros((remaining_batch_size, len(img_idx), y, z, 3))
            batch_labels = np.zeros((remaining_batch_size, 5))
            for folder in range(remaining_batch_size):
                imgs = os.listdir(
                    source_path
                    + "/"
                    + t[folder + (num_batches * batch_size)].split(";")[0]
                )
                for idx, item in enumerate(img_idx):
                    image = imread(
                        source_path
                        + "/"
                        + t[folder + (num_batches * batch_size)].strip().split(";")[0]
                        + "/"
                        + imgs[item]
                    ).astype(np.float32)
                    resized_image = resize(image, (y, z))
                    normalized_image = resized_image / 255.0

                    batch_data[folder, idx, :, :, 0] = normalized_image[:, :, 0]
                    batch_data[folder, idx, :, :, 1] = normalized_image[:, :, 1]
                    batch_data[folder, idx, :, :, 2] = normalized_image[:, :, 2]

                batch_labels[
                    folder,
                    int(t[folder + (num_batches * batch_size)].strip().split(";")[2]),
                ] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.


In [6]:
curr_dt_time = datetime.datetime.now()
train_path = (
    "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/train"
)
val_path = "/home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/val"
num_train_sequences = len(train_doc)
print("# training sequences =", num_train_sequences)
num_val_sequences = len(val_doc)
print("# validation sequences =", num_val_sequences)
num_epochs = 15  # choose the number of epochs
print("# epochs =", num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 15


## Model

Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.


In [7]:
from keras.models import Sequential, Model
from keras.layers import (
    Dense,
    GRU,
    Flatten,
    TimeDistributed,
    Flatten,
    BatchNormalization,
    Dropout,
    Activation,
)
from keras.layers import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


# write your model here
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(
        Conv3D(64, (3, 3, 3), padding="same", activation="elu", input_shape=input_shape)
    )
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding="same"))

    model.add(Conv3D(128, (3, 3, 3), padding="same", activation="elu"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding="same"))

    model.add(Conv3D(256, (3, 3, 3), padding="same", activation="elu"))
    model.add(MaxPooling3D(pool_size=(2, 2, 2), padding="same"))

    model.add(Flatten())
    model.add(Dense(256, activation="elu"))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation="elu"))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation="softmax"))

    return model


y = 45
z = 45
# Assuming input shape and number of classes
input_shape = (10, y, z, 3)
num_classes = 5  # Number of gesture classes

# Create the model
model = create_model(input_shape, num_classes)

2024-03-02 08:16:29.418078: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-02 08:16:29.418852: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-02 08:16:29.419436: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.


In [8]:
optimiser = "adam"  # write your optimizer
model.compile(
    optimizer=optimiser,
    loss="categorical_crossentropy",
    metrics=["categorical_accuracy"],
)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 10, 45, 45, 64)    5248      
                                                                 
 max_pooling3d (MaxPooling3  (None, 5, 23, 23, 64)     0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 5, 23, 23, 128)    221312    
                                                                 
 max_pooling3d_1 (MaxPoolin  (None, 3, 12, 12, 128)    0         
 g3D)                                                            
                                                                 
 conv3d_2 (Conv3D)           (None, 3, 12, 12, 256)    884992    
                                                                 
 max_pooling3d_2 (MaxPoolin  (None, 2, 6, 6, 256)      0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.


In [9]:
train_generator = generator(train_path, train_doc, batch_size, y, z)
val_generator = generator(val_path, val_doc, batch_size, y, z)

In [10]:
model_name = (
    "model_init" + "_" + str(curr_dt_time).replace(" ", "").replace(":", "_") + "/"
)

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = (
    model_name
    + "model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5"
)

checkpoint = ModelCheckpoint(
    filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    period=1,
)

LR = ReduceLROnPlateau(
    monitor="val_loss", factor=0.2, patience=5, min_lr=0.001
)  # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.


In [11]:
if (num_train_sequences % batch_size) == 0:
    steps_per_epoch = int(num_train_sequences / batch_size)
else:
    steps_per_epoch = (num_train_sequences // batch_size) + 1

if (num_val_sequences % batch_size) == 0:
    validation_steps = int(num_val_sequences / batch_size)
else:
    validation_steps = (num_val_sequences // batch_size) + 1

print(steps_per_epoch)
print(validation_steps)

15
3


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.


In [12]:
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=num_epochs,
    verbose=1,
    callbacks=callbacks_list,
    validation_data=val_generator,
    validation_steps=validation_steps,
    class_weight=None,
    workers=1,
    initial_epoch=0,
)

Source path =  /home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/train ; batch size = 45


/tmp/ipykernel_22860/2132049712.py:32: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(


Epoch 1/15


2024-03-02 08:16:35.462660: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-02 08:16:35.586119: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-02 08:16:36.376621: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-02 08:16:38.533828: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fcb74b19060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-02 08:16:38.533860: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1050 Ti, Compute Capability 6.1
2024-03-02 08:16:38.544871: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1709347598.667012   22916 device_compil

13/15 [=========================>....] - ETA: 5s - loss: 4.7325 - categorical_accuracy: 0.2068

/tmp/ipykernel_22860/2132049712.py:66: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(


15/15 [==============================] - ETA: 0s - loss: 4.4553 - categorical_accuracy: 0.2157Source path =  /home/ron/Documents/Learning/Upgrad/Neural_Network_Project/Project_data/val ; batch size = 45

Epoch 1: saving model to model_init_2024-03-0208_16_29.142857/model-00001-4.45532-0.21569-2.28921-0.22000.h5


/home/ron/.local/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


15/15 [==============================] - 64s 4s/step - loss: 4.4553 - categorical_accuracy: 0.2157 - val_loss: 2.2892 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 2/15
15/15 [==============================] - ETA: 0s - loss: 2.3229 - categorical_accuracy: 0.2112
Epoch 2: saving model to model_init_2024-03-0208_16_29.142857/model-00002-2.32286-0.21116-1.48274-0.42000.h5
15/15 [==============================] - 48s 3s/step - loss: 2.3229 - categorical_accuracy: 0.2112 - val_loss: 1.4827 - val_categorical_accuracy: 0.4200 - lr: 0.0010
Epoch 3/15
15/15 [==============================] - ETA: 0s - loss: 1.6976 - categorical_accuracy: 0.3454
Epoch 3: saving model to model_init_2024-03-0208_16_29.142857/model-00003-1.69756-0.34540-1.42610-0.39000.h5
15/15 [==============================] - 44s 3s/step - loss: 1.6976 - categorical_accuracy: 0.3454 - val_loss: 1.4261 - val_categorical_accuracy: 0.3900 - lr: 0.0010
Epoch 4/15
15/15 [==============================] - ETA: 0s - loss: 1.33